In [1]:
import json
import pandas as pd
import numpy as np

In [3]:
with open("model_results.txt", "r") as inf:
    data = inf.read()
    
data = json.loads(data)
probs = []
for d1 in data:
    for d2 in d1:
        for d3 in d2:
            for d4 in d3:
                probs.append(d4)
print(len(probs))

# Both -- both, single, exhaust
# Not -- exhaust, single, both

plist = []
for x in probs:
    params = x[0].split("_")
    plist.append((float(params[0]), float(params[1]), float(params[2]), float(params[3]), x[1]['probs'][1],
                  x[2]['probs'][0], x[2]['probs'][1], 0, 0, x[3]['probs'][1], x[3]['probs'][0]))

import pandas as pd
df = pd.DataFrame(plist, columns=["prior", "alpha", "c1", "c2", "lstnr-both",
                                  "spkr-ab-both", "spkr-ab-single", "spkr-ab-exhaust",
                                 "spkr-anb-both", "spkr-anb-single", "spkr-anb-exhaust"])
#df.to_csv('model_results.csv', index=False)

48000


In [4]:
data = df
data["idx"] = data["alpha"].map(str) + "_" + data["c1"].map(str) + "_" + data["c2"].map(str)
data = data.pivot(index='idx', columns='prior', values=["lstnr-both", "spkr-ab-both", "spkr-ab-single", "spkr-ab-exhaust", "spkr-anb-both", "spkr-anb-single", "spkr-anb-exhaust"])
data.columns.set_levels(['15', '16', '18', '20', '22', '24'],level=1,inplace=True)
data["condition"] = data.index.map(str).to_series()
data[['alpha','c1', 'c2']] = data['condition'].str.split('_',expand=True)
data.head()


lstnr-both                                                    \
prior                  15        16        18        20        22        24   
idx                                                                           
0.01_0.01_0.01   0.437965  0.641409  0.751033  0.753419  0.804533  0.834453   
0.01_0.01_1.0    0.436747  0.640266  0.750102  0.752494  0.803749  0.833764   
0.01_0.01_10.0   0.425981  0.630072  0.741750  0.744194  0.796695  0.827552   
0.01_0.01_11.0   0.424819  0.628962  0.740834  0.743284  0.795920  0.826868   
0.01_0.01_12.0   0.423664  0.627855  0.739922  0.742377  0.795147  0.826186   

               spkr-ab-both                                ...   \
prior                    15        16        18        20  ...    
idx                                                        ...    
0.01_0.01_0.01     0.502037  0.501088  0.500694  0.500686  ...    
0.01_0.01_1.0      0.502037  0.501088  0.500694  0.500686  ...    
0.01_0.01_10.0     0.502037  0.501088  0.500694  0.500686  ...    
0.01_0.01_11.0     0.502037  0.501088  0.500694  0.500686  ...    
0.01_0.01_12.0     0.502037  0.501088  0.500694  0.500686  ...    

               spkr-anb-exhaust                                          \
prior                        15        16        18        20        22   
idx                                                                       
0.01_0.01_0.01         0.501417  0.502534  0.503441  0.503465  0.504042   
0.01_0.01_1.0          0.498942  0.500059  0.500966  0.500990  0.501567   
0.01_0.01_10.0         0.476459  0.477574  0.478479  0.478503  0.479079   
0.01_0.01_11.0         0.473965  0.475080  0.475984  0.476008  0.476584   
0.01_0.01_12.0         0.471473  0.472587  0.473491  0.473514  0.474090   

                               condition alpha    c1    c2  
prior                 24                                    
idx                                                         
0.01_0.01_0.01  0.504454  0.01_0.01_0.01  0.01  0.01  0.01  
0.01_0.01_1.0   0.501979   0.01_0.01_1.0  0.01  0.01   1.0  
0.01_0.01_10.0  0.479491  0.01_0.01_10.0  0.01  0.01  10.0  
0.01_0.01_11.0  0.476996  0.01_0.01_11.0  0.01  0.01  11.0  
0.01_0.01_12.0  0.474502  0.01_0.01_12.0  0.01  0.01  12.0  

[5 rows x 46 columns]

In [10]:
# Here we can do the MSE calculations:

# Human listener values
human_listenr = np.array([[0.3621399, 0.3300654, 0.4022989, 0.5317460, 0.4027778, 0.6451613],]*8000).transpose()


# Human listener & speaker values
human_speaker_listener = np.array([[0.6451613, 0.4027778, 0.5317460, 0.4022989, 0.3300654, 0.3621399, 0.15004748,
                                    0.16481481, 0.17948718, 0.19876784, 0.22018799, 0.29857111, 0.67722524,
                                    0.81111111, 0.82051282, 0.61345020, 0.66414843, 0.61517868, 0.17272727,
                                    0.02407407, 0.00000000, 0.18778195, 0.11566358, 0.08625021, 0.13914190,
                                    0.25375373, 0.07719356, 0.15000932, 0.05753676, 0.04402069, 0.22203940,
                                    0.25854606, 0.35044084, 0.25072983, 0.17316176, 0.34522732, 0.63881870,
                                    0.48770021, 0.57236560, 0.59926086, 0.76930147, 0.61075199],]*8000).transpose()


def mse(array1, array2):
    return np.square(np.array(array1) - np.array(array2)).mean(axis=0)

#data["mse_listener_spkr"] = mse(data.iloc[:, 0:42].transpose(), human_speaker_listener)
data["mse_listener"] = mse(np.array([data["lstnr-both", "15"], data["lstnr-both", "16"], data["lstnr-both", "18"], data["lstnr-both", "20"], data["lstnr-both", "22"], data["lstnr-both", "24"]]), human_listenr)
                           

data['c1'] = data['c1'].astype(float)
data['c2'] = data['c2'].astype(float)
data = data[data['c2']/2 <= data['c1']]

"""
I didn't actually save this dataframe anywhere. I just sorted by the "mse_listener" column to get the best performing model
and then used its 'idx' value to figure out which rsa parameters gave the best results.
"""

data = data.sort_values(by='mse_listener', ascending=True)
#data.sort_values(by='mse_listener_spkr', ascending=False)


data.head(10)


lstnr-both                                                    \
prior                  15        16        18        20        22        24   
idx                                                                           
1.0_2.0_4.0      0.381025  0.607468  0.731731  0.734447  0.792688  0.826814   
0.01_10.0_20.0   0.426593  0.630608  0.742165  0.744606  0.797034  0.827843   
1.0_3.0_5.0      0.414874  0.627693  0.742869  0.745380  0.799212  0.830763   
0.01_9.0_18.0    0.427645  0.631616  0.742997  0.745433  0.797739  0.828466   
0.01_10.0_19.0   0.427701  0.631665  0.743034  0.745470  0.797770  0.828492   
0.01_11.0_20.0   0.427758  0.631714  0.743072  0.745508  0.797800  0.828518   
0.01_8.0_16.0    0.428716  0.632640  0.743841  0.746272  0.798455  0.829097   
0.01_9.0_17.0    0.428766  0.632683  0.743874  0.746305  0.798481  0.829120   
0.01_10.0_18.0   0.428816  0.632726  0.743907  0.746337  0.798507  0.829142   
0.01_11.0_19.0   0.428866  0.632769  0.743939  0.746370  0.798534  0.829165   

               spkr-ab-both                                   ...       \
prior                    15        16        18        20     ...        
idx                                                           ...        
1.0_2.0_4.0        0.235937  0.174384  0.152863  0.152453     ...        
0.01_10.0_20.0     0.477078  0.476131  0.475738  0.475730     ...        
1.0_3.0_5.0        0.102010  0.072100  0.062250  0.062066     ...        
0.01_9.0_18.0      0.479574  0.478626  0.478233  0.478225     ...        
0.01_10.0_19.0     0.477078  0.476131  0.475738  0.475730     ...        
0.01_11.0_20.0     0.474584  0.473637  0.473245  0.473237     ...        
0.01_8.0_16.0      0.482070  0.481122  0.480729  0.480721     ...        
0.01_9.0_17.0      0.479574  0.478626  0.478233  0.478225     ...        
0.01_10.0_18.0     0.477078  0.476131  0.475738  0.475730     ...        
0.01_11.0_19.0     0.474584  0.473637  0.473245  0.473237     ...        

               spkr-anb-exhaust                                          \
prior                        16        18        20        22        24   
idx                                                                       
1.0_2.0_4.0            0.048509  0.068262  0.068873  0.085230  0.099013   
0.01_10.0_20.0         0.452701  0.453600  0.453623  0.454195  0.454605   
1.0_3.0_5.0            0.018410  0.026244  0.026490  0.033140  0.038857   
0.01_9.0_18.0          0.457661  0.458561  0.458585  0.459158  0.459568   
0.01_10.0_19.0         0.455180  0.456079  0.456103  0.456676  0.457085   
0.01_11.0_20.0         0.452701  0.453600  0.453623  0.454195  0.454605   
0.01_8.0_16.0          0.462629  0.463531  0.463554  0.464128  0.464539   
0.01_9.0_17.0          0.460144  0.461045  0.461069  0.461642  0.462053   
0.01_10.0_18.0         0.457661  0.458561  0.458585  0.459158  0.459568   
0.01_11.0_19.0         0.455180  0.456079  0.456103  0.456676  0.457085   

                     condition alpha    c1    c2 mse_listener  
prior                                                          
idx                                                            
1.0_2.0_4.0        1.0_2.0_4.0   1.0   2.0   4.0     0.068658  
0.01_10.0_20.0  0.01_10.0_20.0  0.01  10.0  20.0     0.074018  
1.0_3.0_5.0        1.0_3.0_5.0   1.0   3.0   5.0     0.074100  
0.01_9.0_18.0    0.01_9.0_18.0  0.01   9.0  18.0     0.074426  
0.01_10.0_19.0  0.01_10.0_19.0  0.01  10.0  19.0     0.074445  
0.01_11.0_20.0  0.01_11.0_20.0  0.01  11.0  20.0     0.074463  
0.01_8.0_16.0    0.01_8.0_16.0  0.01   8.0  16.0     0.074841  
0.01_9.0_17.0    0.01_9.0_17.0  0.01   9.0  17.0     0.074858  
0.01_10.0_18.0  0.01_10.0_18.0  0.01  10.0  18.0     0.074874  
0.01_11.0_19.0  0.01_11.0_19.0  0.01  11.0  19.0     0.074891  

[10 rows x 47 columns]

In [ ]:
data = data[]